https://medium.com/dunder-data/create-a-bar-chart-race-animation-in-python-with-matplotlib-477ed1590096
https://stackoverflow.com/questions/28931224/adding-value-labels-on-a-matplotlib-bar-chart
https://github.com/dexplo/bar_chart_race/blob/master/bar_chart_race/_make_chart.py
- brew install ffmpeg

In [1]:
# Standard libraries
from pathlib import Path

# Third-party libraries
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/local/bin/ffmpeg'

In [2]:
data_path = Path('data/')
colors = plt.cm.Dark2(range(6))

In [3]:
trivia_data = pd.read_csv(data_path/'trivia_test.csv', index_col='Date', parse_dates=['Date'])
trivia_data

,Round,Cindy_Yeager_Emily,Connie_Gordon,Jeff_Beth_Jenny,Natalie_Matt,Rick_Debbie,Ryan_Erin
Date,,,,,,,
2020-06-12,Rnd_1,0,2.0,10.0,6,0,10
2020-06-12,Rnd_2,0,0.0,12.0,0,12,8
2020-06-12,Rnd_3,0,0.0,10.0,10,6,6
2020-06-12,Halftime,0,13.0,6.0,2,11,4
2020-06-12,Rnd_4,0,10.0,18.0,10,10,15
2020-06-12,Rnd_5,0,15.0,15.0,0,10,10
2020-06-12,Rnd_6,0,16.5,16.5,15,15,15
2020-06-12,Final,0,0.0,-22.0,-25,-25,-25
2020-06-12,Winners,0,20.0,25.0,0,0,15


In [4]:
trivia_cumsum = trivia_data.cumsum()
trivia_cumsum['Round'] = trivia_data['Round']
trivia_cumsum

,Round,Cindy_Yeager_Emily,Connie_Gordon,Jeff_Beth_Jenny,Natalie_Matt,Rick_Debbie,Ryan_Erin
Date,,,,,,,
2020-06-12,Rnd_1,0,2.0,10.0,6,0,10
2020-06-12,Rnd_2,0,2.0,22.0,6,12,18
2020-06-12,Rnd_3,0,2.0,32.0,16,18,24
2020-06-12,Halftime,0,15.0,38.0,18,29,28
2020-06-12,Rnd_4,0,25.0,56.0,28,39,43
2020-06-12,Rnd_5,0,40.0,71.0,28,49,53
2020-06-12,Rnd_6,0,56.5,87.5,43,64,68
2020-06-12,Final,0,56.5,65.5,18,39,43
2020-06-12,Winners,0,76.5,90.5,18,39,58


In [5]:
def nice_axes(ax):
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

In [6]:
def prepare_data(df, steps=50):
    df = df.reset_index()
    df.index = df.index * steps
    last_idx = df.index[-1] + 1
    df_expanded = df.reindex(range(last_idx))
    df_expanded['Date'] = df_expanded['Date'].fillna(method='ffill')
    df_expanded['Round'] = df_expanded['Round'].fillna(method='ffill')
    df_expanded = df_expanded.set_index('Date')
    df_rank_expanded = df_expanded.rank(axis=1, method='first')
    df_expanded = df_expanded.interpolate()
    df_rank_expanded = df_rank_expanded.interpolate()
    
    return df_expanded, df_rank_expanded

df_expanded, df_rank_expanded = prepare_data(trivia_cumsum, steps=10)

In [7]:
df_expanded.head()

,Round,Cindy_Yeager_Emily,Connie_Gordon,Jeff_Beth_Jenny,Natalie_Matt,Rick_Debbie,Ryan_Erin
Date,,,,,,,
2020-06-12,Rnd_1,0.0,2.0,10.0,6.0,0.0,10.0
2020-06-12,Rnd_1,0.0,2.0,11.2,6.0,1.2,10.8
2020-06-12,Rnd_1,0.0,2.0,12.4,6.0,2.4,11.6
2020-06-12,Rnd_1,0.0,2.0,13.6,6.0,3.6,12.4
2020-06-12,Rnd_1,0.0,2.0,14.8,6.0,4.8,13.2


In [8]:
df_rank_expanded.head()

,Cindy_Yeager_Emily,Connie_Gordon,Jeff_Beth_Jenny,Natalie_Matt,Rick_Debbie,Ryan_Erin
Date,,,,,,
2020-06-12,1.0,3.0,5.0,4.0,2.0,6.0
2020-06-12,1.0,2.9,5.1,3.9,2.2,5.9
2020-06-12,1.0,2.8,5.2,3.8,2.4,5.8
2020-06-12,1.0,2.7,5.3,3.7,2.6,5.7
2020-06-12,1.0,2.6,5.4,3.6,2.8,5.6


In [34]:
from matplotlib.animation import FuncAnimation

def init():
    ax.clear()
    nice_axes(ax)
    ax.set_ylim(.2, 6.8)

def update(i):
    for bar in ax.containers:
        bar.remove()
    for text in ax.texts[:]:
        text.remove()

    labels = df_rank_expanded.columns
    y = df_rank_expanded.iloc[i]
    width = df_expanded.iloc[i].drop('Round').astype('float64')
    ax.barh(y=y, width=width, color=colors, tick_label=labels, height=0.6)

    ax.set_title('GameNight Trivia Cummulative Stats', fontsize='smaller')
    ax.tick_params(axis='y', labelsize=8)

    # Date box (and roun)
    date_str = df_expanded.index[i].strftime('%Y-%m-%d')
    label    = f"{date_str} {df_expanded.iloc[i]['Round']}"
    props = dict(boxstyle='round', facecolor='0.8', alpha=0.5)
    ax.text(0.65, 0.08, label, transform=ax.transAxes, fontsize=12,
        verticalalignment='top', bbox=props, clip_on=True)


    n_bars = 6
    bar_location = df_rank_expanded.iloc[i].values
    top_filt = (bar_location > 0) & (bar_location < n_bars + 1)
    bar_location = bar_location[top_filt]
    bar_length = df_expanded.iloc[i].drop('Round').astype('float64').values[top_filt]

    zipped = zip(bar_length, bar_location)

    # Number of points between bar and label
    space = 0.01

    # Vertical alignment for position vals
    ha = 'left'

    for x1,y1 in zipped:
        # Use x va as label and format number with one decimal place
        label = f'{x1:,.1f}'

        xtext, ytext = ax.transLimits.transform((x1, y1))
        xtext += space

        xtext, ytext = ax.transLimits.inverted().transform((xtext, ytext))
        ax.text(xtext, ytext, label, ha=ha, va='center', clip_on=True)

# dpi is dots per inch
fig = plt.Figure(figsize=(15, 7), dpi=70)
ax = fig.add_subplot()
anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(df_expanded), 
                     interval=100, repeat=False)

In [35]:
from IPython.display import HTML
html = anim.to_html5_video()
HTML(html)

In [278]:
df_expanded.tail()

,Round,Cindy_Yeager_Emily,Connie_Gordon,Jeff_Beth_Jenny,Natalie_Matt,Rick_Debbie,Ryan_Erin
Date,,,,,,,
2020-06-26,Final,145.0,197.9,149.5,99.0,160.4,132.0
2020-06-26,Final,145.5,198.3,149.5,99.0,160.8,132.0
2020-06-26,Final,146.0,198.7,149.5,99.0,161.2,132.0
2020-06-26,Final,146.5,199.1,149.5,99.0,161.6,132.0
2020-06-26,Winners,147.0,199.5,149.5,99.0,162.0,132.0


In [36]:
anim.save(data_path/'trivia_stats_race.mp4')

# Todos
- Decrease spacing between bars
- Add data labels to outside of bars